In [ ]:
import pandas as pd
import numpy as np

# This simulates loading your data.
# In your real code, you would use: df = pd.read_csv("your_file.csv")
data = {
    'วันครบอายุเห็นชอบ': [
        '2/6/2562', '4/19/2562', '2/6/2562', '___', '5/20/2562', '6/10/2562',
        '6/24/2562', '8/3/2562', '9/1/2562', '9/4/2562', '11/4/2562', '10/19/2562',
        '9/24/2562', '11/29/2562', '11/27/2562', '12/7/2562', '3/30/2563', '4/7/2563',
        '5/1/2563', '4/20/2563', '4/17/2563', '5/5/2563', '5/24/2563', '7/14/2563',
        '6/5/2563', '7/6/2563', 'nan', '8/10/2563', '9/4/2563', '8/10/2563', 'nan',
        '10/10/2563', '9/30/2563', '12/9/2563', '1/11/2564', '1/20/2564', '2/14/2564',
        '2/27/2564', '4/4/2564', '3/3/2564', '4/21/2564', '5/17/2564', '5/22/2564',
        '5/17/2564', '7/11/2564', '10/7/2564', '11/26/2564', '3/30/2564', '1/26/2565',
        '20/03/65', '25/03/65', '29/03/65', '3/5/2565', '6/6/2565', 'รายใหม่',
        '11/6/2565', '18/07/2565', '26/07/2565', '7/8/2565', '-', '24/09/2565',
        '30/09/2565', '30/11/2565', '26/12/2565', '08/02/2561 - 07/02/2566',
        '13/06/2561 - 12/06/2566', '25/06/2561 - 24/06/2566', '19/07/2561 - 18/07/2566',
        '24/07/2561 - 23/07/2566', 'nan', '24/08/2561 - 23/08/2566', '03/09/2561 - 02/09/2566',
        '28/09/2561 - 27/09/2566', '21/10/2561 - 20/10/2566', '03/12/2561 - 02/12/2566',
        '22/12/2561 - 21/12/2566', 'nan', '20/04/2562 - 19/04/2567', '16/05/2562 - 15/05/2567',
        '06/06/2562 - 05/06/2567', '21/05/2562 - 20/05/2567', 'nan', '30/07/2562 - 29/07/2567',
        '02/09/2562 - 01/09/2567', 'nan', '05/09/2562 - 04/09/2567', '20/10/2562 - 19/10/2567',
        '31/10/2562 - 30/10/2567', '30/11/2562 - 29/11/2567', '12/12/2562 - 11/12/2567',
        'nan', '06/02/2563 - 05/02/2568', '31/03/2563 - 30/03/2568', '22/04/2563 - 21/04/2568',
        '08/04/2563 - 07/04/2568', 'nan', '06/05/2563 - 05/05/2568', 'nan', 'nan',
        '31/07/2563 - 30/07/2568', '02/07/2563 - 01/07/2568', 'nan', 'nan', 'nan', 'nan',
        'nan', 'nan', 'nan', 'nan', 'nan'
    ]
}
df = pd.DataFrame(data)

def clean_and_split_dates_to_ad(df, col_name='วันครบอายุเห็นชอบ'):
    s = df[col_name].astype(str).str.strip()
    mask = s.str.contains(' - ', na=False)
    
    split_data = s[mask].str.split(' - ', n=1, expand=True)
    
    df['วันที่อนุญาต'] = pd.Series(dtype='object')
    if not split_data.empty:
        df.loc[mask, 'วันที่อนุญาต'] = split_data[0]
        df.loc[mask, col_name] = split_data[1]

    def convert_thai_dates_to_ad(series):
        dates = series.astype(str).str.strip().replace(['___', '-', 'nan', 'รายใหม่'], np.nan)
        clean_dates = dates.dropna()
        
        if clean_dates.empty:
            return pd.Series(pd.NaT, index=series.index)
            
        parts = clean_dates.str.split('/', expand=True)
        year = pd.to_numeric(parts[parts.columns[-1]], errors='coerce')
        ad_year = np.where(year < 100, year + 2500, year) - 543
        
        temp_df = pd.DataFrame({
            'year': ad_year,
            'month': pd.to_numeric(parts[1], errors='coerce'),
            'day': pd.to_numeric(parts[0], errors='coerce')
        })
        
        converted_dates = pd.to_datetime(temp_df, errors='coerce')
        return converted_dates.reindex(series.index)

    df['วันที่อนุญาต'] = convert_thai_dates_to_ad(df['วันที่อนุญาต'])
    df[col_name] = convert_thai_dates_to_ad(df[col_name])
    
    return df

def convert_ad_to_be_str(ad_date_series):
    # This function converts a pandas Series of AD datetime objects to BE date strings
    be_series = ad_date_series.copy()
    valid_dates_mask = be_series.notna()
    
    # Apply conversion only on valid (non-NaT) dates
    be_series.loc[valid_dates_mask] = (
        be_series.loc[valid_dates_mask] + pd.DateOffset(years=543)
    ).dt.strftime('%d/%m/%Y')
    
    return be_series

# --- Main Execution ---

# 1. Clean data and convert dates to AD for processing
df_processed = clean_and_split_dates_to_ad(df.copy())

# 2. Create a new DataFrame for export
df_for_export = df_processed.copy()

# 3. Convert date columns back to BE format for the export file
df_for_export['วันที่อนุญาต'] = convert_ad_to_be_str(df_for_export['วันที่อนุญาต'])
df_for_export['วันครบอายุเห็นชอบ'] = convert_ad_to_be_str(df_for_export['วันครบอายุเห็นชอบ'])

# 4. Define the output filename
output_filename = "FA_Data_Cleaned_BE.xlsx"

# 5. Save the final DataFrame (with BE dates) to an Excel file
df_for_export.to_excel(
    output_filename,
    sheet_name='Cleaned Data (BE)',
    index=False
)

print(f"Data successfully cleaned and saved to '{output_filename}' with dates in BE format.")

Data successfully cleaned and saved to 'FA_Data_Cleaned.xlsx'


### FA-2 Cleaning 

In [9]:
import pandas as pd

input_file = "Dataset\FA-2 (ปี 2565).xlsm"
output_file = "FA-2 Sheet.xlsx"

data = pd.read_excel(input_file, dtype=str)

p_columns = ["รายใหม่", "ต่ออายุ"]
is_p_or_1 = data[p_columns].isin(["P", "1"])
mask = is_p_or_1.any(axis=1)

data.loc[mask, 'ประเภทคำขอ'] = is_p_or_1[mask].idxmax(axis=1)

data.to_excel(
    output_file,
    sheet_name='Processed_Data',
    index=False
)

In [1]:
import pandas as pd
import numpy as np

input_file = "Dataset\FA-2 (ปี 2565).xlsm"
output_file = "FA-2_Processed.xlsx"

df = pd.read_excel(input_file, dtype=str)

df['สถิติ'] = pd.to_datetime(df["วันที่ยื่นคำขอ"], errors='coerce').dt.quarter.apply(
    lambda q: f'Quarter {int(q)}' if pd.notna(q) else ''
)

app_type_cols = df.columns.intersection(["รายใหม่", "ต่ออายุ"])
if not app_type_cols.empty:
    is_p_or_1 = df[app_type_cols].isin(["P", "1"])
    mask = is_p_or_1.any(axis=1)
    df.loc[mask, 'ประเภทคำขอ'] = is_p_or_1[mask].idxmax(axis=1)

cols = df.columns.tolist()
try:
    # Find insertion point after the column containing 'FA-2'
    ref_col_index = next(i for i, col in enumerate(cols) if 'FA-2' in col)
    # Move 'สถิติ' column to the correct position
    cols.insert(ref_col_index + 1, cols.pop(cols.index('สถิติ')))
    df = df[cols]
except (StopIteration, ValueError):
    # If reference column isn't found, the order remains as is
    pass

df.to_excel(output_file, sheet_name='Processed_Data', index=False)

C:\Users\kongl\AppData\Local\Temp\ipykernel_22724\774214242.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['สถิติ'] = pd.to_datetime(df["วันที่ยื่นคำขอ"], errors='coerce').dt.quarter.apply(


In [2]:
import pandas as pd
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
import time
import random

API_KEY = "sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh"
BASE_URL = "https://api.opentyphoon.ai/v1"
MODEL = "typhoon-v2.1-12b-instruct"
FILE_PATH = "Dataset\FA-2 Sheet.xlsx"
OUTPUT_FILE_PATH = "Dataset\FA-2 Sheet_classified_llm_final.xlsx"
COMPANY_COLUMN = 'ชื่อบริษัท FA '
PREFIX_COLUMN = 'คำนำหน้า'
VALID_TYPES = {'บจก', 'บล', 'ธนาคาร', 'บลจ'}
SYSTEM_PROMPT = (
    "คุณคือ AI ผู้เชี่ยวชาญการจำแนกประเภทบริษัทในไทย "
    "ให้ตอบกลับมาเพียงคำเดียวจากตัวเลือกนี้: 'บจก.', 'บล.', 'ธนาคาร', 'บลจ.' "
    "ถ้าไม่เข้าข่าย ให้ตอบ 'Unknown'"
)

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

def classify(company_name):
    if pd.isna(company_name):
        return ""
    for i in range(5):
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"'{company_name}'"}
            ],
            max_tokens=10,
            temperature=0.0
        )
        result = response.choices[0].message.content.strip().replace('.', '')
        if result in VALID_TYPES:
            return result
        if i < 4:
            time.sleep((2 ** i) + random.uniform(0, 1))
    return "Unknown"

df = pd.read_excel(FILE_PATH)

if PREFIX_COLUMN not in df.columns:
    df[PREFIX_COLUMN] = ""

names = df[COMPANY_COLUMN].dropna().unique().tolist()

with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(classify, names))

mapping = dict(zip(names, results))
df[PREFIX_COLUMN] = df[COMPANY_COLUMN].map(mapping).fillna('')
df.to_excel(OUTPUT_FILE_PATH, index=False)



ประมวลผลเสร็จสิ้น บันทึกไฟล์ที่: Dataset\FA-2 Sheet_classified_llm_final.xlsx

ตัวอย่างผลลัพธ์:
                       ชื่อบริษัท FA  คำนำหน้า
0                    เคที ซีมิโก้ บล.       บล
1       เมอร์ชั่น พาร์ทเนอร์ บล. บมจ.       บล
2              แอพเพิล เวลธ์ บล. บมจ.       บล
3              เพลินจิต แคปปิตอล บจก.      บจก
4             เวลแคป แอดไวเซอรี่ บจก.      บจก
5             เอส 14 แอดไวเซอรี่ บจก.      บจก
6                     เออีซี บล. บมจ.       บล
7            ออพท์เอเชีย แคปิตอล บจก.      บจก
8        หยวนต้า (ประเทศไทย) บล. บมจ.       บล
9        หยวนต้า (ประเทศไทย) บล. บมจ.       บล
10                  โกลเบล็ก บล. บจก.       บล
11                    กรุงไทย ธ. บมจ.   ธนาคาร
12        แอสเซท โปร แมเนจเม้นท์ บจก.      บจก
13  แคปปิตอล ลิ้งค์ แอ๊ดไวเซอรี่ บจก.      บจก
14         ที่ปรึกษา เอเชีย พลัส บจก.      บจก
15  แคปปิตอล ลิ้งค์ แอ๊ดไวเซอรี่ บจก.      บจก
16        เคทีบี (ประเทศไทย) บล. บมจ.       บล
17                    กรุงเทพ ธ. บมจ.   ธนาคาร
18        

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import asyncio
from tqdm.asyncio import tqdm as anic_tqdm
import nest_asyncio

nest_asyncio.apply()

API_KEY = ("sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh")
FILE_PATH = ("Dataset\FA-2 (ปี 2565)(Sheet1).csv")
OUTPUT_PATH = "FA-2565_cleaned_and_exploded_minimal.csv"
TARGET_COLUMN_NAME = "ให้ความเห็นชอบผู้ควบคุมฯ (แบบ FA-2)"
SYSTEM_PROMPT = "You are an efficient data cleaning assistant. Your task is to extract only Thai human names. Return ONLY the names, each on a new line. Provide no other text or formatting."
USER_PROMPT_TEMPLATE = "Extract all human names from this text:\n\n---\n{}\n---"

client = OpenAI(api_key=API_KEY, base_url="https://api.opentyphoon.ai/v1")

async def extract_names_async(text_to_clean: str) -> list[str]:
    if pd.isna(text_to_clean) or not str(text_to_clean).strip():
        return []
    
    response = await client.chat.completions.create(
        model="typhoon-v2.1-12b-instruct",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT_TEMPLATE.format(text_to_clean)}
        ],
        max_tokens=32784,
        temperature=0.1,
        timeout=30.0
    )
    return [name.strip() for name in response.choices[0].message.content.split('\n') if name.strip()]

async def main():
    df = pd.read_csv(FILE_PATH)
    tasks = [extract_names_async(text) for text in df[TARGET_COLUMN_NAME]]
    extracted_names = await anic_tqdm.gather(*tasks, desc="Processing names")

    (df.assign(cleaned_names=extracted_names)
       .explode('cleaned_names')
       .assign(**{TARGET_COLUMN_NAME: lambda x: x['cleaned_names'].fillna('')})
       .drop(columns=['cleaned_names'])
       .to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig'))

    print(f"Process complete. Output saved to {OUTPUT_PATH}")

if __name__ == "__main__":
    asyncio.run(main())

Processing names:   0%|          | 0/517 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-3' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at c:\Users\kongl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\asyncio.py:75> exception=BadRequestError("Error code: 400 - {'detail': {'error': 'litellm.BadRequestError: OpenAIException - Input validation error: `inputs` tokens + `max_new_tokens` must be <= 4049. Given: 102 `inputs` tokens and 8000 `max_new_tokens`', 'context': {'status_code': 400}}}")>
Traceback (most recent call last):
  File "c:\Users\kongl\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\kongl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\asyncio.py", line 76, in wrap_awaitable
    return i, await f
              ^^^^^^^
  File "C:\Users\kongl\AppData\Local\Tem